## ДЗ

Постройте топ фильмов в категориях Action и Comedy

1) считываем все файлы

In [2]:
import pandas as pd

In [4]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [5]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [103]:
movies.head()

,movieId,title,genres,Action,Comedy
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,,Comedy
1,2,Jumanji (1995),Adventure|Children|Fantasy,,
2,3,Grumpier Old Men (1995),Comedy|Romance,,Comedy
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,Comedy
4,5,Father of the Bride Part II (1995),Comedy,,Comedy


In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


2) нужно сделать метку Action & Comedy, сделаем просто два столбца в котторых будем проверять наличия этого слова в тегах

In [50]:
movies[['Action', 'Comedy']]= movies.apply\
(lambda x: pd.Series(['Action' if ('Action' in x['genres']) else '', \
                      'Comedy' if ('Comedy' in x['genres']) else '']), axis=1)

In [53]:
movies.head(10) # проверяем сработало ли, всё ок

,movieId,title,genres,Action,Comedy
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,,Comedy
1,2,Jumanji (1995),Adventure|Children|Fantasy,,
2,3,Grumpier Old Men (1995),Comedy|Romance,,Comedy
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,Comedy
4,5,Father of the Bride Part II (1995),Comedy,,Comedy
5,6,Heat (1995),Action|Crime|Thriller,Action,
6,7,Sabrina (1995),Comedy|Romance,,Comedy
7,8,Tom and Huck (1995),Adventure|Children,,
8,9,Sudden Death (1995),Action,Action,
9,10,GoldenEye (1995),Action|Adventure|Thriller,Action,


Моя стратегия состоит в том, чтобы взять среднюю оценку среди топа 1000 по количеству поставленных оценок

In [150]:
top_1000 = ratings.groupby('movieId').count().sort_values('userId', ascending=False)[:1000]
top_1000.columns = ['userId','rating_cnt','timestamp'] # даём уникальные названия, иначе при сортировке будут проблемы
all_rating = ratings.groupby('movieId').mean()
all_rating.columns = ['userId','rating_mean','timestamp']


In [151]:
join_table = top_1000.join(all_rating, on=None,
    how='left',
    lsuffix='movieId',
    rsuffix='movieId',
    sort=False,
) # джойним таблицу топ 1000 с рейтингом
final_table = join_table.join(movies, on=None,
    how='left',
    lsuffix='movieId',
    rsuffix='movieId',
    sort=False,
)

In [156]:
final_table[final_table['Action']=='Action'].sort_values('rating_mean', ascending=False)[['title', 'rating_mean']].head(20)

,title,rating_mean
movieId,,
318,I Love Trouble (1994),4.429022
898,Star Wars: Episode V - The Empire Strikes Back...,4.310345
246,New York Cop (Nyû Yôku no koppu) (1993),4.293103
858,Escape from New York (1981),4.289062
3275,"10th Victim, The (La decima vittima) (1965)",4.220930
1208,Kull the Conqueror (1997),4.219626
296,Virtuosity (1995),4.197068
2571,Teenage Mutant Ninja Turtles II: The Secret of...,4.192446
1228,"Assignment, The (1997)",4.187500


In [157]:
final_table[final_table['Comedy']=='Comedy'].sort_values('rating_mean', ascending=False)[['title', 'rating_mean']].head(20)

,title,rating_mean
movieId,,
318,I Love Trouble (1994),4.429022
1204,Leave It to Beaver (1997),4.300000
750,"Pompatus of Love, The (1996)",4.268041
904,Withnail & I (1987),4.261905
1267,"Truman Show, The (1998)",4.250000
1213,"Kiss Me, Guido (1997)",4.250000
1197,In the Company of Men (1997),4.232394
3275,"10th Victim, The (La decima vittima) (1965)",4.220930
2329,Babes in Toyland (1934),4.217054
